In [6]:
from glob import iglob
import os
from time import sleep


from joblib import Parallel, delayed
import pandas as pd
import screed
import seaborn as sns
from tqdm import tqdm

In [10]:
cd /mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019


In [11]:
ls -lha

total 2.6G
drwxr-xr-x 5 olga root 4.0K Jan 10 15:34 ./
drwxr-xr-x 3 olga root 4.0K Dec 25 17:48 ../
drwxr-xr-x 5 olga czb  4.0K Dec 26 19:44 Archaea/
drwxr-xr-x 5 olga czb   16K Dec 26 19:44 Bacteria/
drwxr-xr-x 9 olga czb   32K Jan 10 15:35 Eukaryota/
-rw-r--r-- 1 olga czb  754K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv
-rw-r--r-- 1 olga czb   68K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv.gz
-rw-r--r-- 1 olga czb  133K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.parquet
-rw-r--r-- 1 olga czb  4.5K Jan 10 15:34 human_visual_transduction_with_uniprot_ids.csv
-rw-r--r-- 1 olga czb  1.2K Jan 10 15:34 human_visual_transduction_with_uniprot_ids.csv.gz
-rw-r--r-- 1 olga czb  3.7K Jan 10 15:34 human_visual_transduction_with_uniprot_ids.parquet
-rw-r--r-- 1 olga czb   64M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara.csv
-rw-r--r-- 1 olga czb  1.8M Jan 10 08:02 opisthokont_not_human_transcription_factors_ensembl_c

# Read human visual transduction genes

In [33]:
visual = pd.read_csv("human_visual_transduction_with_uniprot_ids.csv")
print(visual.shape)
visual.head()

(3053, 3)


,source__uniprot_id,source__id_type,source__db_id
0,P29973,UniProtKB-ID,CNGA1_HUMAN
1,P29973,Gene_Name,CNGA1
2,P29973,Gene_Synonym,CNCG
3,P29973,Gene_Synonym,CNCG1
4,P29973,GI,2506302


# Go to Quest for Orthologs fastas

In [34]:
cd /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019


In [35]:
ls -lha

total 2.6G
drwxr-xr-x 5 olga root 4.0K Jan 10 15:34 ./
drwxr-xr-x 3 olga root 4.0K Dec 25 17:48 ../
drwxr-xr-x 5 olga czb  4.0K Dec 26 19:44 Archaea/
drwxr-xr-x 5 olga czb   16K Dec 26 19:44 Bacteria/
drwxr-xr-x 9 olga czb   32K Jan 10 15:35 Eukaryota/
-rw-r--r-- 1 olga czb  754K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv
-rw-r--r-- 1 olga czb   68K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv.gz
-rw-r--r-- 1 olga czb  133K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.parquet
-rw-r--r-- 1 olga czb   76K Jan 15 10:57 human_visual_transduction_with_uniprot_ids.csv
-rw-r--r-- 1 olga czb   19K Jan 15 10:57 human_visual_transduction_with_uniprot_ids.csv.gz
-rw-r--r-- 1 olga czb   28K Jan 15 10:57 human_visual_transduction_with_uniprot_ids.parquet
-rw-r--r-- 1 olga czb   64M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara.csv
-rw-r--r-- 1 olga czb  1.8M Jan 10 08:02 opisthokont_not_human_transcription_factors_ensembl_c

## Read species metadata

In [36]:
species_metadata = pd.read_csv("species_metadata.csv")
print(species_metadata.shape)
species_metadata.head()

(78, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
1,UP000000798,224324,AQUAE,1553,0,1557,Aquifex aeolicus (strain VF5),4290.0,Aquifex aeolicus,strain VF5
2,UP000006548,3702,ARATH,27475,14123,41920,Arabidopsis thaliana (Mouse-ear cress),1496.0,Arabidopsis thaliana,Mouse-ear cress
3,UP000001570,224308,BACSU,4260,7,4268,Bacillus subtilis (strain 168),4290.0,Bacillus subtilis,strain 168
4,UP000001414,226186,BACTN,4782,0,4823,Bacteroides thetaiotaomicron (strain ATCC 2914...,4290.0,Bacteroides thetaiotaomicron,strain ATCC 29148 / DSM 2079 / NCTC 10582 / E5...


### Subset to opisthokonts

In [37]:
# Estimated opisthokonta divergence time from http://timetree.org/
t = 1105
opisthokonts = species_metadata.query('divergence_from_human_mya <= @t')
print(opisthokonts.shape)
opisthokonts.head()

(35, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
5,UP000007241,684364,BATDJ,8610,0,8685,Batrachochytrium dendrobatidis (strain JAM81 /...,1105.0,Batrachochytrium dendrobatidis,strain JAM81 / FGSC 10211
6,UP000009136,9913,BOVIN,23774,14534,38438,Bos taurus (Bovine),96.0,Bos taurus,Bovine
8,UP000001554,7739,BRAFL,28542,2,28608,Branchiostoma floridae (Florida lancelet) (Amp...,684.0,Branchiostoma floridae,Florida lancelet
9,UP000001940,6239,CAEEL,19986,8309,28507,Caenorhabditis elegans,797.0,Caenorhabditis elegans,NaN


In [38]:
opisthokonts.query('scientific_name == "Homo sapiens"')

,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
32,UP000005640,9606,HUMAN,21146,74769,96332,Homo sapiens (Human),0.0,Homo sapiens,Human


In [39]:
opisthokonts_not_human = opisthokonts.query('scientific_name != "Homo sapiens"')
print(opisthokonts_not_human.shape)
opisthokonts_not_human.head()

(34, 10)


,proteome_id,tax_id,oscode,n_main_fasta_canonical,n_additional_fasta_isoforms,n_gene2acc,species_name,divergence_from_human_mya,scientific_name,common_name_or_strain
0,UP000007062,7165,ANOGA,12553,971,13619,Anopheles gambiae (African malaria mosquito),797.0,Anopheles gambiae,African malaria mosquito
5,UP000007241,684364,BATDJ,8610,0,8685,Batrachochytrium dendrobatidis (strain JAM81 /...,1105.0,Batrachochytrium dendrobatidis,strain JAM81 / FGSC 10211
6,UP000009136,9913,BOVIN,23774,14534,38438,Bos taurus (Bovine),96.0,Bos taurus,Bovine
8,UP000001554,7739,BRAFL,28542,2,28608,Branchiostoma floridae (Florida lancelet) (Amp...,684.0,Branchiostoma floridae,Florida lancelet
9,UP000001940,6239,CAEEL,19986,8309,28507,Caenorhabditis elegans,797.0,Caenorhabditis elegans,NaN


# Use ENSEMBL Rest API to get homologous TFs

## Actually run it

In [40]:
visual.head()

,source__uniprot_id,source__id_type,source__db_id
0,P29973,UniProtKB-ID,CNGA1_HUMAN
1,P29973,Gene_Name,CNGA1
2,P29973,Gene_Synonym,CNCG
3,P29973,Gene_Synonym,CNCG1
4,P29973,GI,2506302


In [41]:
visual.sample(10)

,source__uniprot_id,source__id_type,source__db_id
2055,P08100,GI,1236137
564,A0A1B0GVU3,CRC64,89611212D656D06D
1609,P62873,neXtProt,NX_P62873
963,Q9UMX6,GeneReviews,GUCA1B
2765,P51160,BioMuta,PDE6C
2339,C9J7V6,Ensembl,ENSG00000133256
2998,A6PVH5,CRC64,176DA167CA68CA96
2564,P43080,GI,623405
1078,Q9UBF6,EMBL-CDS,-
1120,Q9UBF6,CRC64,CE1E6CAC940C8257


### Separate out samples with ensg ids and not

In [45]:
ensg_rows = visual.source__db_id.str.startswith("ENSG0")
visual_non_ensg = visual.loc[~ensg_rows]
print(visual_non_ensg.shape)
visual_non_ensg.head()

(2981, 3)


,source__uniprot_id,source__id_type,source__db_id
0,P29973,UniProtKB-ID,CNGA1_HUMAN
1,P29973,Gene_Name,CNGA1
2,P29973,Gene_Synonym,CNCG
3,P29973,Gene_Synonym,CNCG1
4,P29973,GI,2506302


In [46]:
visual_ensg = visual.loc[ensg_rows]
print(visual_ensg.shape)
visual_ensg.head()

(72, 3)


,source__uniprot_id,source__id_type,source__db_id
42,P29973,Ensembl,ENSG00000198515
119,P16520,Ensembl,ENSG00000111664
212,P11488,Ensembl,ENSG00000114349
283,Q02846,Ensembl,ENSG00000132518
370,P35913,Ensembl,ENSG00000133256


In [47]:
from urllib.error import HTTPError

In [52]:

dfs = []



def single_homology_to_series(homology, ignore_fields=['align_seq', 'cigar_line', 'target', 'source']):

    homology_for_series = {}
    for key, value in homology.items():
        if not key in ignore_fields:
            homology_for_series[key] = value
        if key in ('target', 'source'):
            for k, v in value.items():
                if not k in ignore_fields:
                    homology_for_series[f"{key}__{k}"] = v

    series = pd.Series(homology_for_series)
    return series


for ensembl_id in tqdm(visual_ensg.source__db_id):
    sleep(1)
    try:
        response = ensembl_rest.homology_ensemblgene(ensembl_id, target_taxon=opisthokonts_not_human.tax_id, cigar_line=False, sequence=None)
    except HTTPError:
        # Probably a 503 error, meaning server is busy, so wait 2 seconds and try again
        sleep(2)
        response = ensembl_rest.homology_ensemblgene(ensembl_id, target_taxon=opisthokonts_not_human.tax_id, cigar_line=False, sequence=None)

    data = response['data']
    homologies = data[0]['homologies']
    df = pd.DataFrame(map(single_homology_to_series, homologies))
    dfs.append(df)
visual_opsithokonts_ensembl = pd.concat(dfs, ignore_index=True)
print(visual_opsithokonts_ensembl.shape)
visual_opsithokonts_ensembl.head()


100%|██████████| 72/72 [09:49<00:00,  8.19s/it]

(17828, 16)



<ipython-input-52-1b665e2cc1da>:33: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  visual_opsithokonts_ensembl = pd.concat(dfs, ignore_index=True)


,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,target__perc_pos,target__protein_id,target__species,target__taxon_id,taxonomy_level,type
0,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,50.5929,62.8458,ENSP00000384264,homo_sapiens,9606,ENSG00000183862,57.8313,71.8373,ENSP00000328478,homo_sapiens,9606,Bilateria,other_paralog
1,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,18.1818,35.9684,ENSP00000384264,homo_sapiens,9606,ENSG00000055118,11.9068,23.5548,ENSP00000262186,homo_sapiens,9606,Bilateria,other_paralog
2,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,17.3913,35.1779,ENSP00000384264,homo_sapiens,9606,ENSG00000143630,17.0543,34.4961,ENSP00000357342,homo_sapiens,9606,Bilateria,other_paralog
3,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,55.0725,67.0619,ENSP00000384264,homo_sapiens,9606,ENSG00000144191,60.2305,73.3429,ENSP00000272602,homo_sapiens,9606,Bilateria,other_paralog
4,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,24.7694,40.8432,ENSP00000384264,homo_sapiens,9606,ENSG00000170289,23.2386,38.3189,ENSP00000316605,homo_sapiens,9606,Bilateria,other_paralog


In [53]:
visual_opsithokonts_ensembl

,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,target__perc_pos,target__protein_id,target__species,target__taxon_id,taxonomy_level,type
0,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,50.5929,62.8458,ENSP00000384264,homo_sapiens,9606,ENSG00000183862,57.8313,71.8373,ENSP00000328478,homo_sapiens,9606,Bilateria,other_paralog
1,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,18.1818,35.9684,ENSP00000384264,homo_sapiens,9606,ENSG00000055118,11.9068,23.5548,ENSP00000262186,homo_sapiens,9606,Bilateria,other_paralog
2,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,17.3913,35.1779,ENSP00000384264,homo_sapiens,9606,ENSG00000143630,17.0543,34.4961,ENSP00000357342,homo_sapiens,9606,Bilateria,other_paralog
3,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,55.0725,67.0619,ENSP00000384264,homo_sapiens,9606,ENSG00000144191,60.2305,73.3429,ENSP00000272602,homo_sapiens,9606,Bilateria,other_paralog
4,NaN,ENSEMBL_PARALOGUES,ENSG00000198515,24.7694,40.8432,ENSP00000384264,homo_sapiens,9606,ENSG00000170289,23.2386,38.3189,ENSP00000316605,homo_sapiens,9606,Bilateria,other_paralog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17823,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000114349,88.8571,95.7143,ENSP00000232461,homo_sapiens,9606,ENSSTUG00000023497,88.8571,95.7143,ENSSTUP00000055369,salmo_trutta,8032,Gnathostomata,ortholog_one2many
17824,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000114349,91.4286,96.5714,ENSP00000232461,homo_sapiens,9606,ENSMALG00000001144,91.4286,96.5714,ENSMALP00000001558,monopterus_albus,43700,Gnathostomata,ortholog_one2one
17825,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000114349,76.0000,80.5714,ENSP00000232461,homo_sapiens,9606,ENSCSEG00000000424,90.7850,96.2457,ENSCSEP00000000602,cynoglossus_semilaevis,244447,Gnathostomata,ortholog_one2one
17826,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000114349,30.8571,33.4286,ENSP00000232461,homo_sapiens,9606,ENSPMAG00000004273,87.8049,95.1219,ENSPMAP00000004667,petromyzon_marinus,7757,Vertebrata,ortholog_one2one


In [56]:
visual_opsithokonts_ensembl.target__species.value_counts() 

homo_sapiens                    815
sinocyclocheilus_rhinocerous    183
cyprinus_carpio_germanmirror    178
cyprinus_carpio_huanghe         176
cyprinus_carpio_hebaored        174
                               ... 
sorex_araneus                    37
ciona_savignyi                   35
chrysolophus_pictus              35
choloepus_hoffmanni              23
saccharomyces_cerevisiae          6
Name: target__species, Length: 240, dtype: int64

In [57]:
visual_opsithokonts_ensembl.query('target__species == "saccharomyces_cerevisiae"')

,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,target__perc_pos,target__protein_id,target__species,target__taxon_id,taxonomy_level,type
667,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000111664,40.8824,61.1765,ENSP00000229264,homo_sapiens,9606,YOR212W,32.8605,49.1726,YOR212W,saccharomyces_cerevisiae,4932,Opisthokonta,ortholog_one2many
2197,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000111664,40.8824,61.1765,ENSP00000229264,homo_sapiens,9606,YOR212W,32.8605,49.1726,YOR212W,saccharomyces_cerevisiae,4932,Opisthokonta,ortholog_one2many
7038,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000111664,40.8824,61.1765,ENSP00000229264,homo_sapiens,9606,YOR212W,32.8605,49.1726,YOR212W,saccharomyces_cerevisiae,4932,Opisthokonta,ortholog_one2many
12119,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000111664,40.8824,61.1765,ENSP00000229264,homo_sapiens,9606,YOR212W,32.8605,49.1726,YOR212W,saccharomyces_cerevisiae,4932,Opisthokonta,ortholog_one2many
12542,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000111664,40.8824,61.1765,ENSP00000229264,homo_sapiens,9606,YOR212W,32.8605,49.1726,YOR212W,saccharomyces_cerevisiae,4932,Opisthokonta,ortholog_one2many
17483,NaN,ENSEMBL_ORTHOLOGUES,ENSG00000111664,40.8824,61.1765,ENSP00000229264,homo_sapiens,9606,YOR212W,32.8605,49.1726,YOR212W,saccharomyces_cerevisiae,4932,Opisthokonta,ortholog_one2many


The only gene here is [STE4 in Yeast](https://www.yeastgenome.org/locus/S000005738) which corresponds to [GNB3 in human](https://uswest.ensembl.org/Homo_sapiens/Gene/Summary?g=ENSG00000111664;r=12:6839954-6847393)

# Write to disk!

In [29]:
# tfs_opisthokonts = pd.concat([tfs_opsithokonts_symbol, tfs_opsithokonts_ensembl])

In [54]:
visual_opsithokonts_ensembl.to_csv('opisthokont_not_human_visual_system_ensembl_compara.csv', index=False)
visual_opsithokonts_ensembl.to_parquet('opisthokont_not_human_visual_system_ensembl_compara.parquet', index=False)

In [32]:
pwd

'/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019'

In [ ]:
ls -lha